# 第10章 性能評価

## 10.3 LLMを用いた自動評価

### 10.3.2 Japanese Vicuna QA Benchmarkによる自動評価

#### 環境の準備

In [ ]:
!pip install bitsandbytes datasets transformers[torch,sentencepiece] openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 91.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached 

In [ ]:
from transformers.trainer_utils import set_seed

# 乱数シードを42に固定する
set_seed(42)

#### データセットの準備

In [ ]:
from datasets import load_dataset

# データセットを読み込む
test_dataset = load_dataset(
    "llm-book/ja-vicuna-qa-benchmark", split="test"
)
print(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for llm-book/ja-vicuna-qa-benchmark contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/llm-book/ja-vicuna-qa-benchmark.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question_id', 'category', 'turns'],
    num_rows: 80
})


In [ ]:
# データを表示する
test_data = test_dataset[0]
print(test_data)

{'question_id': 1, 'category': 'generic', 'turns': ['時間管理能力を向上させるにはどうしたらいいですか？']}


#### パイプラインの作成

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

model_name = "tokyotech-llm/Swallow-7b-instruct-v0.1"
# AutoTokenizerでトークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_name)
# モデルを量子化して読み込むためのパラメータを指定する
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
# 生成を行うモデルであるAutoModelForCausalLMを使ってモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

# パイプラインを作成する
generation_config = {
    "do_sample": True,
    "max_new_tokens": 2048,
    "temperature": 0.99,
    "top_p": 0.95
}
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config
)

#### 質問に回答するためのプロンプトの作成

In [ ]:
prompt_template = "以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。\n\n### 指示:\n{instruction}\n\n### 応答:\n"
# プロンプトテンプレートの{instruction}に入力テキストに置換する
prompt = prompt_template.format(instruction=test_data["turns"][0])
print(prompt)

以下に、あるタスクを説明する指示があります。リクエストを適切に完了するための回答を記述してください。

### 指示:
時間管理能力を向上させるにはどうしたらいいですか？

### 応答:



#### 評価対象LLMによる質問の回答の生成

In [ ]:
# 質問の回答を生成する
output = text_generation_pipeline(prompt)
# プロンプト部分を削除して回答のみにする
generated_text_swallow = output[0]["generated_text"].replace(
    prompt, ""
)
print(generated_text_swallow)

時間管理を向上させる方法には、以下のような方法が考えられます。

1. **目標を設定する**: タスクを時間内に完了させるためには、目標を設定することが重要です。目標は具体的で測定可能である必要があります。例えば、1時間内に3つのタスクを完了させることを目標として設定できます。
2. **優先順位を決める**: タスクには優先順位をつけることで、時間の有効的な使い方ができます。優先順位は、タスクの重要性や緊急性に基づいて決めることをお勧めします。
3. **時間を分割する**: タスクを時間内に完了させるためには、タスクを小さな時間に分割して行うことが重要です。例えば、3時間の時間を30分の時間に分割してタスクを完了させることができます。
4. **タイムトラッキングを行う**: タイムトラッキングは、タスクにかかる時間を測定することで、時間の有効的な使い方ができます。タイムトラッキングには様々な方法がありますが、例えばタイマーを使用することで、タスクにかかる時間を測定できます。
5. **時間を有効に使う**: 時間を有効に使うためには、時間の使い方に集中することが重要です。例えば、作業中にメールやメッセージなどの通知をオフにすることで、時間を有効に使うことができます。

時間管理能力を向上させることで、タスクの完了率や生産性が上がり、仕事や生活において大きな影響を与えることができます。


#### 評価者LLMによる単一採点の実施

In [ ]:
from pprint import pprint

# 単一採点のためのプロンプトを作成する
single_judge_prompt_template = "[インストラクション]\n以下に示されるユーザの質問に対してAIアシスタントが提供した回答の質を評価してください。具体的には、回答の有用性、関連性、正確性、深さ、創造性、詳細レベルなどの要素を考慮して評価してください。評価の際には、まず回答内容を簡単に、できるだけ客観的に説明してください。説明を行った後、必ず「[[rating]]」という形式で、回答を1から10の尺度で評価してください（例：[[5]]）。\".\n\n[ユーザの質問]\n{question}\n\n[アシスタントの答えの始まり]\n{answer}\n[アシスタントの答えの終わり]"

# OpenAI APIに渡す入力を作成する
messages = [
    {
        "role": "system",
        "content": "あなたは役に立つアシスタントです。",
    },
    {
        "role": "user",
        "content": single_judge_prompt_template.format(
            question=test_data["turns"][0],
            answer=generated_text_swallow,
        ),
    },
]
pprint(messages)

[{'content': 'あなたは役に立つアシスタントです。', 'role': 'system'},
 {'content': '[インストラクション]\n'
             '以下に示されるユーザの質問に対してAIアシスタントが提供した回答の質を評価してください。具体的には、回答の有用性、関連性、正確性、深さ、創造性、詳細レベルなどの要素を考慮して評価してください。評価の際には、まず回答内容を簡単に、できるだけ客観的に説明してください。説明を行った後、必ず「[[rating]]」という形式で、回答を1から10の尺度で評価してください（例：[[5]]）。".\n'
             '\n'
             '[ユーザの質問]\n'
             '時間管理能力を向上させるにはどうしたらいいですか？\n'
             '\n'
             '[アシスタントの答えの始まり]\n'
             '時間管理を向上させる方法には、以下のような方法が考えられます。\n'
             '\n'
             '1. **目標を設定する**: '
             'タスクを時間内に完了させるためには、目標を設定することが重要です。目標は具体的で測定可能である必要があります。例えば、1時間内に3つのタスクを完了させることを目標として設定できます。\n'
             '2. **優先順位を決める**: '
             'タスクには優先順位をつけることで、時間の有効的な使い方ができます。優先順位は、タスクの重要性や緊急性に基づいて決めることをお勧めします。\n'
             '3. **時間を分割する**: '
             'タスクを時間内に完了させるためには、タスクを小さな時間に分割して行うことが重要です。例えば、3時間の時間を30分の時間に分割してタスクを完了させることができます。\n'
             '4. **タイムトラッキングを行う**: '
             'タイムトラッキングは、タスクにかかる時間を測定することで、時間の有効的な使い方ができます。タイムトラッキングには様々な方

取得したOpenAI_API_KEYを入力してください

In [ ]:
%env OPENAI_API_KEY=sk-...

In [ ]:
from openai import OpenAI

# 評価者LLMとしてGPT-4を用いて、単一採点による評価を行う
# リクエストのパラメータを準備
params = {
    "messages": messages,
    "max_tokens": 2048,
    "model": "gpt-4-turbo-2024-04-09"
}
# OpenAI APIのクライアントを初期化
client = OpenAI()
# OpenAI APIにリクエストを送信
response = client.chat.completions.create(**params)
# レスポンスからLLMの応答を取得
content = response.choices[0].message.content
print(content)

アシスタントの回答は、時間管理能力を向上させるための各種具体的な手法を提供しています。指摘した方法は実際に取り入れやすく、日常的に実施することが可能です。また、それぞれの方法について短い説明が追加されており、ユーザーが理解しやすい形で説明されています。

以下の点で評価します：
- **有用性**: 提案された各手法は効果的で実用的であり、時間管理に役立つと考えられる。
- **関連性**: 問いに対する直接的な回答であり、時間管理能力向上のための適切なアドバイスが含まれています。
- **正確性**: 記載されている方法は一般的な時間管理技術であり、誤った情報は含まれていません。
- **深さ**: 基本的な時間管理の技術が網羅されていますが、より高度なテクニックや心理学的アプローチについての言及はありません。
- **創造性**: 提案されている手法は比較的一般的ですが、効果的な時間管理のための実用的なアドバイスであるため問題はない。
- **詳細レベル**: 各手法には簡潔ながらも必要な説明が付与されていますが、より具体的な例や実際の適用例については述べられていません。

全体的に回答は質問に対して適切で有益な指南を提供しているため、高い評価をします。ただし、創造性や深さの面でさらに改善の余地がある可能性が考えられます。

[[rating]]: 8


#### GPT3.5よる質問の回答の生成

In [ ]:
# GPT-3.5で質問の回答を生成する
messages = [
    {
        "role": "system",
        "content": "あなたは役に立つアシスタントです。"
    },
    {"role": "user", "content": prompt},
]
# リクエストのパラメータを準備
params = {
    "messages": messages,
    "max_tokens": 2048,
    "model": "gpt-3.5-turbo-0125",
}
# OpenAI APIにリクエストを送信
response = client.chat.completions.create(**params)
# レスポンスからLLMの応答を取得
generated_text_gpt_3_5 = response.choices[0].message.content
print(generated_text_gpt_3_5)

時間管理能力を向上させるためにいくつかの方法があります。まず、To-Doリストやカレンダーを使って予定やタスクを整理し、優先順位をつけることが重要です。また、タイマーやアラームを使って時間を区切り、集中して作業することも効果的です。さらに、作業を途中で中断することなく、完了することが大切です。不必要な時間を無駄に過ごさないように、時間の使い方を常に意識することも重要です。


#### ペア比較のためのプロンプトの作成

In [ ]:
# ペア比較のためのプロンプトを作成する
pair_judge_prompt_template = "[ユーザーの質問]\n{question}\n\n[アシスタントAの答えの始まり]\n{answer_a}\n[アシスタントAの答えの終わり]\n\n[アシスタントBの答えの始まり]\n{answer_b}\n[アシスタントBの答えの終わり]"

# OpenAI APIに渡す入力を作成
messages = [
    {
        "role": "system",
        "content": (
            "以下に示されるユーザの質問に対して2人のAIアシスタントが提供した回答の質を評価してください。"
            "回答の内容がユーザの指示に従っており、"
            "ユーザの質問によりよく答えているアシスタントを選んでください。"
            "具体的には、回答の有用性、関連性、正確性、深さ、創造性、"
            "詳細レベルなどの要素を考慮する必要があります。"
            "評価の際には、まず2つの回答を比較し、"
            "簡単な説明をしてください。立場が偏らないようにし、"
            "回答の提示順があなたの判断に影響しないようにしてください。"
            "回答の長さが評価に影響しないこと、"
            "特定のアシスタントの名前を好まないこと、"
            "できるだけ客観的であること、に気をつけてください。"
            "説明の後に、"
            "最終的な判断を以下の形式に従って出力してください：アシスタントAが優れていれば[[A]]、"
            "アシスタントBが優れていれば[[B]]、同点の場合は[[C]]"
        ),
    },
    {
        "role": "user",
        "content": pair_judge_prompt_template.format(
            question=test_data["turns"][0],
            answer_a=generated_text_swallow,
            answer_b=generated_text_gpt_3_5,
        ),
    },
]
pprint(messages)

[{'content': '以下に示されるユーザの質問に対して2人のAIアシスタントが提供した回答の質を評価してください。回答の内容がユーザの指示に従っており、ユーザの質問によりよく答えているアシスタントを選んでください。具体的には、回答の有用性、関連性、正確性、深さ、創造性、詳細レベルなどの要素を考慮する必要があります。評価の際には、まず2つの回答を比較し、簡単な説明をしてください。立場が偏らないようにし、回答の提示順があなたの判断に影響しないようにしてください。回答の長さが評価に影響しないこと、特定のアシスタントの名前を好まないこと、できるだけ客観的であること、に気をつけてください。説明の後に、最終的な判断を以下の形式に従って出力してください：アシスタントAが優れていれば[[A]]、アシスタントBが優れていれば[[B]]、同点の場合は[[C]]',
  'role': 'system'},
 {'content': '[ユーザーの質問]\n'
             '時間管理能力を向上させるにはどうしたらいいですか？\n'
             '\n'
             '[アシスタントAの答えの始まり]\n'
             '時間管理を向上させる方法には、以下のような方法が考えられます。\n'
             '\n'
             '1. **目標を設定する**: '
             'タスクを時間内に完了させるためには、目標を設定することが重要です。目標は具体的で測定可能である必要があります。例えば、1時間内に3つのタスクを完了させることを目標として設定できます。\n'
             '2. **優先順位を決める**: '
             'タスクには優先順位をつけることで、時間の有効的な使い方ができます。優先順位は、タスクの重要性や緊急性に基づいて決めることをお勧めします。\n'
             '3. **時間を分割する**: '
             'タスクを時間内に完了させるためには、タスクを小さな時間に分割して行うことが重要です。例えば、3時間の時間を30分の時間に分割してタスクを完了させることができます。\n'
             '4. 

#### 評価者LLMによるペア比較の実施

In [ ]:
# 評価者LLMとしてGPT-4を用いて、ペア比較による評価を行う
params = {
    "messages": messages,
    "max_tokens": 2048,
    "model": "gpt-4-turbo-2024-04-09",
}
# OpenAI APIにリクエストを送信
response = client.chat.completions.create(**params)
# レスポンスからLLMの応答を取得
content = response.choices[0].message.content
print(content)

アシスタントAとアシスタントBの回答を比較すると、両者は時間管理の向上に有効な具体的な方法を提案していますが、アシスタントAの方がより詳細で体系的な回答を提供しています。

アシスタントAは具体的なステップや例を提示しており、各アドバイスを詳しく述べています。例えば、時間の分割の利点を説明する際に、「3時間の仕事を30分に分割する」という具体例を挙げており、理解しやすいです。また、タイムトラッキングの具体的な方法（タイマーの使用）も紹介しています。

一方で、アシスタントBも有益な情報を提供していますが、アシスタントAと比較するとやや概括的であり、具体的な行動指針や例が少ないです。アシスタントBはTo-Doリストやカレンダーの使用、タイマーやアラームの利用を推奨していますが、これらのツールをどのように効果的に使用するかについての詳細は提供されていません。

全体として、アシスタントAの回答は時間管理技術を向上させるための具体的かつ実践的なステップをより詳細に説明しており、ユーザーが実際に適用できる具体的なアドバイスが含まれているため、より優れた回答と判断できます。

したがって、アシスタントAが優れていると結論付けます。[[A]]
